In [ ]:
# 1. 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 2. 모델 설정
class myCNN(nn.Module): # nn.Module 상속받음
    def __init__(self):
        super(myCNN, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(1, 16, 5 , padding=2), # 28×28 -> 28×28
            nn.ReLU(),

            nn.Conv2d(16, 32, 5, padding=2), # 28×28 -> 28×28
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 28×28 -> 14×14

            nn.Conv2d(32, 64, 5, padding=2), # 14×14 -> 14×14
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 14×14 -> 7×7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        out = self.cnn_layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

In [ ]:
# 3. 하이퍼파라미터 설정
learning_rate = 0.01
num_epoch = 50
batch_size = 100

In [ ]:
# 3. 데이터 전처리 및 로더 설정
transform = transforms.Compose([
    transforms.ToTensor(),  # [0, 255] -> [0.0, 1.0]
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 55.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.70MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.06MB/s]


In [ ]:
# 4. loss 함수 및 optimizer 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = myCNN().to(device)

# loss function, optimizer 선언
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# 5. Accuracy 함수 정의
def EstimateAccuracy(dloader, imodel):
    correct = 0
    total = 0
    for image, label in dloader:
        x = image.to(device)  ## back prop. 안하고, forward prop. 만하겠다.
        y_ = label.to(device)

        output = imodel.forward(x)
        _, output_index= torch.max(output, 1)

        total += label.size(0)
        correct += (output_index== y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))
    return 100*correct/total

In [ ]:
# 6. 훈련
loss_arr= []
accu_arr= []
for epoch in range(num_epoch):
    for image, label in train_loader:  # batch_size만큼
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output=model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        accuracy = EstimateAccuracy(test_loader, model)
        loss_arr.append(loss.item())
        accu_arr.append(accuracy)


Epoch 0, Loss: 0.4261
Accuracy of Test Data: 93.9000015258789
